In [92]:
import bitarray
import struct
import numpy as np
def print_hex_bytes(src):
    for i in range(8):
        str = src[i*8:(i+1)*8]
        print(' '.join('{:02X}'.format(a) for a in str))

In [93]:
file = open("test.txt", mode="rb")
words = np.fromfile(file, dtype=np.uint64)
file.close()
block = words[:16]
answer = np.zeros_like(words)
Nw = 16
kNw = np.array(0, dtype=np.uint64)
print(kNw)
#kNw = 0
Nr = 80
t = np.zeros(3, dtype=np.uint8)
subKeys = np.zeros((Nr//4+1, Nw), dtype=np.uint64)
C240 = np.array(0x1bd11bdaa9fc1a22, np.uint64)

t[0] = 38
t[1] = 111

R = np.zeros([8,8],  dtype = np.uint64)
R[0] = [24, 13, 8, 47, 8, 17, 22, 37]
R[1] = [38, 19, 10, 55, 49, 18, 23, 52]
R[2] = [33, 4, 51, 13, 34, 41, 59, 17]
R[3] = [5, 20, 48, 41, 47, 28, 16, 25]
R[4] = [41, 9, 37, 31, 12, 47, 44, 30]
R[5] = [16, 34, 56, 51, 4, 53, 42, 41]
R[6] = [31, 44, 47, 46, 19, 42, 44, 25]
R[7] = [9, 48, 35, 52, 23, 31, 37, 20] 

pi = np.array([0, 9, 2, 13, 6, 11, 4, 15, 10, 7, 12, 3, 14, 5, 8, 1])
R

0


array([[24, 13,  8, 47,  8, 17, 22, 37],
       [38, 19, 10, 55, 49, 18, 23, 52],
       [33,  4, 51, 13, 34, 41, 59, 17],
       [ 5, 20, 48, 41, 47, 28, 16, 25],
       [41,  9, 37, 31, 12, 47, 44, 30],
       [16, 34, 56, 51,  4, 53, 42, 41],
       [31, 44, 47, 46, 19, 42, 44, 25],
       [ 9, 48, 35, 52, 23, 31, 37, 20]], dtype=uint64)

In [94]:
keys = np.copy(block)
kNw = C240
for i in range(Nw):
    kNw ^= np.uint64(keys[i])

keys = np.append(keys, kNw)

t[2] = t[0] ^ t[1]

keys

array([3290013473472425389, 3070657282893845114, 3290013473472425389,
       3070657282893845114, 3290013473472425389, 3070657282893845114,
       3290013473472425389, 3070657282893845114, 3290013473472425389,
       3070657282893845114, 3290013473472425389, 3070657282893845114,
       3290013473472425389, 3070657282893845114, 3290013473472425389,
       3070657282893845114, 2004413935125273122], dtype=uint64)

In [95]:
for round in range((Nr // 4) + 1):
  for i in range(Nw):
    subKeys[round][i] = keys[(round + i) % (Nw + 1)]
    if Nw - 1:
        subKeys[round][i] += round
    elif Nw - 2:
        subKeys[round][i] += t[(round + 1) % 3]
    elif Nw - 3:
        subKeys[round][i] += t[round % 3]    

In [96]:
def MIX(x0, x1, r):
    y0 = x0 + x1
    y1 = (x1 << np.uint64(r)) | (x1 >> np.uint64(64 - r))
    y1 ^= y0
    return (y0, y1)


In [97]:
def deMIX(x0, x1, r):
    x1 ^= x0
    y1 = (x1 << np.uint64(64 - r)) | (x1 >> np.uint64(r))
    y0 = x0 - y1
    return (y0, y1) 

In [98]:
def crypt(block):
    e = np.zeros(Nw, dtype=np.uint64)
    f = np.zeros(Nw, dtype=np.uint64)
    v = np.copy(block)

    for round in range(Nr):
        if round % 4 == 0:
            s = round // 4
            for i in range(Nw):
                e[i] = v[i] + subKeys[s][i]
        else:
            for i in range(Nw):
                e[i] = v[i]
        
        for i in range(Nw//2):
            x0 = np.copy(e[i*2])
            x1 = np.copy(e[i*2 + 1])
            f[i * 2], f[i * 2 + 1] = MIX(x0, x1, R[round % 8][i])
        for i in range(Nw):
            v[i] = f[p[i]]

    newBlock = np.zeros_like(block)
    for i in range(Nw):
        newBlock[i] = v[i] + subKeys[Nr//4][i]
    return newBlock



In [99]:
def decrypt(block):
    e = np.zeros(Nw, dtype=np.uint64)
    f = np.zeros(Nw, dtype=np.uint64)
    v = np.copy(block)
    
    for round in range(Nr-1, 0-1, -1):
        if round % 4 == 0:
            s = round//4
            for i in range(Nw):
                f[i] = v[i] - subKeys[s][i]
        else:
            for i in range(Nw):
                f[i] = v[i]
        
        for i in range(Nw):
            e[i] = f[p[i]]

        for i in range(Nw//2):
            v[i * 2], v[i * 2 + 1] = deMIX(e[i * 2], e[i * 2 + 1], R[(round-1) % 8][i])

    newBlock = np.zeros_like(block)
    for i in range(Nw):
        newBlock[i] = v[i] - subKeys[0][i]
    return newBlock

In [100]:
block = words[:16]
print(f"Block: ")
print_hex_bytes(block)

crypted = crypt(block)
print(f"Crypted: ")
print_hex_bytes(crypted)

print(f"decrypted block")
print_hex_bytes(decrypt(crypted))


Block: 
2DA879D2A892ADAD 2A9D2A879DDA927A 2DA879D2A892ADAD 2A9D2A879DDA927A 2DA879D2A892ADAD 2A9D2A879DDA927A 2DA879D2A892ADAD 2A9D2A879DDA927A
2DA879D2A892ADAD 2A9D2A879DDA927A 2DA879D2A892ADAD 2A9D2A879DDA927A 2DA879D2A892ADAD 2A9D2A879DDA927A 2DA879D2A892ADAD 2A9D2A879DDA927A






Crypted: 
66680AC4852C7354 599EB18885C7D01A 5937019A149BC056 5CFB18942975756B 18EF74E4F377E47D B07CAA21F1DD75CA 8C52D2BF1064787F 333648C34625CA08
101E517DBE9F6F 6DD17F4BE32E67FE 920BA2B65F2FA177 383937C8CC6B1BF4 26431B49E06F6137 F4DBFF1CE007149B BA1571D41ED5C667 1C0976307656256F






decrypted block
B9FFEC9E3E31459A 1A7F6930DF8F3E16 F59073008F4E364D C5FC72CB42421175 62D350FF730E99ED FC2A274C4F9DC59F 8311481ABE973C58 16775E234F0708B7
3FE9FBC7E22CC06D 893D9645D81804DB 88625CFFD9FD4EE A7368990D1186E41 9D03448244B00465 9C9AA1DFD1FBD49 C7C5E2D9600CF935 5B67D07CD52618BB








C:\Users\Murav\AppData\Local\Temp\ipykernel_2804\4242785412.py:10: RuntimeWarning: overflow encountered in scalar add
  e[i] = v[i] + subKeys[s][i]
C:\Users\Murav\AppData\Local\Temp\ipykernel_2804\4242785412.py:24: RuntimeWarning: overflow encountered in scalar add
  newBlock[i] = v[i] + subKeys[Nr//4][i]
C:\Users\Murav\AppData\Local\Temp\ipykernel_2804\2624641330.py:4: RuntimeWarning: overflow encountered in scalar subtract
  y0 = x0 - y1
C:\Users\Murav\AppData\Local\Temp\ipykernel_2804\45127739.py:10: RuntimeWarning: overflow encountered in scalar subtract
  f[i] = v[i] - subKeys[s][i]
C:\Users\Murav\AppData\Local\Temp\ipykernel_2804\45127739.py:23: RuntimeWarning: overflow encountered in scalar subtract
  newBlock[i] = v[i] - subKeys[0][i]
